# IN4310 - Nore Skulesson Stene - noress - Assignment 1

### Task 1 - Dataset loading

In [2]:
# Imports
import os
import numpy as np
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.pyplot as plt

# Do not know if needed..
#%matplotlib inline

In [3]:
# Set dataset directories
DATASET_ROOT = Path("mandatory1_data/mandatory1_data")
CLASS_FOLDERS = []
OUTPUT_ROOT = Path("./dataset_splits") 

# Create lists to store file paths and labels
image_paths = []
labels = []
class_names = sorted([directory.name for directory in DATASET_ROOT.iterdir() if directory.is_dir()])  # Get class names

# Task 1a - Splitting Data
# Collect all image file paths and their labels
for label, class_name in enumerate(class_names):
    class_directory_path = DATASET_ROOT / class_name
    files = list(class_directory_path.glob("*.jpg"))
    image_paths.extend(files)
    labels.extend([label] * len(files))

# Convert lists to NumPy arrays
image_paths = np.array(image_paths)
labels = np.array(labels)

print(class_names)
print(image_paths[0])
print(len(image_paths))

for c in class_names:
    print(c)

# Defining the split sizes
val_size = 2000  
test_size = 3000  
train_ratio = 1 - ((val_size + test_size) / 17034)  # Remaining percentage for training

# First, split into test alone and train+val together
train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
    image_paths, labels, test_size=test_size, stratify=labels, random_state=42
)

# Spliting up train_val into train and validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_val_paths, train_val_labels, test_size=val_size, stratify=train_val_labels, random_state=42
)

# Function to move files into new directory
def move_files(image_paths, labels, split_name):
    split_dir = OUTPUT_ROOT / split_name
    split_dir.mkdir(parents=True, exist_ok=True)

    for class_name in class_names:
        (split_dir / class_name).mkdir(parents=True, exist_ok=True)  # Creating class folders

    for img_path, label in zip(image_paths, labels):
        class_name = class_names[label]
        shutil.copy(img_path, split_dir / class_name / img_path.name)  # Copying files to new location

# Moving the images to new directories
move_files(train_paths, train_labels, "train")
move_files(val_paths, val_labels, "val")
move_files(test_paths, test_labels, "test")

print(f"Data successfully split into Training ({len(train_paths)}), Validation ({len(val_paths)}), and Test ({len(test_paths)}) sets.")

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
mandatory1_data\mandatory1_data\buildings\0.jpg
17034
buildings
forest
glacier
mountain
sea
street
Data successfully split into Training (12034), Validation (2000), and Test (3000) sets.


In [4]:
# Task 1b
# TODO ... 
def VerifySplits():
    # Read the image paths in each split
    train_files = {img.name for img in train_paths}
    val_files = {img.name for img in val_paths}
    test_files = {img.name for img in test_paths}

    # Checking for overlapping files using intersection
    train_val_overlap = train_files.intersection(val_files)
    train_test_overlap = train_files.intersection(test_files)
    val_test_overlap = val_files.intersection(test_files)

    # Asserting no overlaps exist
    assert not train_val_overlap, f"Overlap found between train and val: {train_val_overlap}"
    assert not train_test_overlap, f"Overlap found between train and test: {train_test_overlap}"
    assert not val_test_overlap, f"Overlap found between val and test: {val_test_overlap}"

    print("Dataset verification complete")

VerifySplits()

Dataset verification complete


In [ ]:
# Task 1c
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


class ImageDataset(Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform

        # Get all image paths and labels
        for class_idx, class_name in enumerate(class_names):
            class_folder = dataset_dir / class_name
            for img_path in class_folder.glob("*.jpg"):  # Adjust if other formats exist
                self.image_paths.append(img_path)
                self.labels.append(class_idx)  # Class index as label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label


# Transforming Data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224 for consistency
    transforms.ToTensor(),          # Convert image to PyTorch tensor (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
])

# Creating Dataloaders
def get_dataloaders(batch_size=32, shuffle=True):
    train_dataset = ImageDataset(OUTPUT_ROOT / "train", transform=transform)
    val_dataset = ImageDataset(OUTPUT_ROOT / "val", transform=transform)
    test_dataset = ImageDataset(OUTPUT_ROOT / "test", transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader


# Testing dataloaders
train_loader, val_loader, test_loader = get_dataloaders()
print(f"DataLoaders created. Train: {len(train_loader.dataset)}, Val: {len(val_loader.dataset)}, Test: {len(test_loader.dataset)}")

"""
# Example: Fetch a single batch of images and labels
images, labels = next(iter(train_loader))
print(f"Batch shape: {images.shape}, Labels: {labels[:8]}")  # Show first 8 labels
"""

DataLoaders created. Train: 12034, Val: 2000, Test: 3000


'\n# Example: Fetch a single batch of images and labels\nimages, labels = next(iter(train_loader))\nprint(f"Batch shape: {images.shape}, Labels: {labels[:8]}")  # Show first 8 labels\n'

### Task 2 - Building and training the ResNet Architecture

In [ ]:
from Precode.ResNet import ResNet
# Maybe make a Config dict, for tweaking parameters ??
# Train data, test on validation set, optimize, check accuracy while training epochs

# Initializing 
model = ResNet(img_channels=3, num_layers=18, num_classes=6)
model.train()

# Creating an instance of "torch.optim.SGD"
optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001)

# Creating loss function - CEL
cel = torch.nn.CrossEntropyLoss()

# Checking if CUDA GPU is available and sending model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model.to(device)

# Training
for batchImages, labels in train_loader:
    batchImages, labels = batchImages.to(device), labels.to(device)  # Corrected variable names

    optimizer.zero_grad()  # Zeroing gradients

    outputs = model(batchImages)  # Forward 
    loss = cel(outputs, labels)  

    loss.backward()  # Backpropagation
    optimizer.step()


# Evaluating
model.eval()
correct = 0
total = 0
with torch.no_grad():  # Disabling gradients
    for valImages, valLabels in val_loader:
        valImages, valLabels = valImages.to(device), valLabels.to(device)
        outputs = model(valImages)

        # Get predicted classes
        _, predicted = torch.max(outputs, 1)
        total += valLabels.size(0)
        correct += (predicted == valLabels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")



Using device: cpu


: 